# UAT-C2-001: Verify Successful MD Run

In [ ]:
import os

import numpy as np
import yaml
from ase.build import bulk
from ase.db import connect

# 1. Create a configuration file for the MD run
config_md = {
    "project_name": "uat_test_cu_md",
    "system": {"elements": ["Cu"], "lattice": "fcc", "num_structures": 1},
    "exploration": {
        "temperature": 500.0,
        "md_steps": 100,
        "mlip_model": "emt",  # Use a fast, built-in potential
    },
    "sampling": {"method": "random", "fraction": 1.0},
}

with open("config_md.yaml", "w") as f:
    yaml.dump(config_md, f)

print("Configuration file `config_md.yaml` created.")

In [ ]:
# 2. Define the initial structure for comparison
initial_atoms = bulk("Cu", "fcc", a=3.6)
initial_positions = initial_atoms.get_positions().copy()

# The EMT calculator needs to be attached to get energy
from ase.calculators.emt import EMT

initial_atoms.calc = EMT()
initial_energy = initial_atoms.get_potential_energy()

print(f"Initial Potential Energy: {initial_energy:.4f} eV")

In [ ]:
# 3. Run the MLIP-AutoPipe pipeline
# NOTE: This assumes 'mlip-autopipec' is an executable in the path.
# You might need to adjust this call depending on your installation.
os.system("mlip-autopipec run --config config_md.yaml")
print("Pipeline execution complete.")

In [ ]:
# 4. Load the results and verify the changes
final_db_path = "uat_test_cu_md.db"
assert os.path.exists(final_db_path), "Output database not found!"

db_final = connect(final_db_path)
final_atoms = db_final.get_atoms(id=1)
final_positions = final_atoms.get_positions()

final_atoms.calc = EMT()
final_energy = final_atoms.get_potential_energy()

print(f"Final Potential Energy: {final_energy:.4f} eV")

# Verify that the atomic positions have changed significantly
position_difference = np.sum((initial_positions - final_positions) ** 2)
print(f"Sum of squared differences in positions: {position_difference}")

assert position_difference > 1e-6, "MD simulation did not change the atomic positions!"
print("\nSuccess: The structure was demonstrably modified by the MD exploration.")